In [1]:
import pandas as pd
import jsonlines
import os
from glob import glob

In [28]:
files = glob("../outputs/*/*/prometheus-*/mt-bench.json")
dfs = []

def filter_score(x):
    if x["score"] > 0:
        return x["score"]
    if "overall score is" in x["critic"]:
        score = int(x["critic"].split("overall score is")[1].split(".")[0].strip().replace(",","").split(" ", 1)[0])
        return score
    elif "a score of " in x["critic"]:
        score = x["critic"].split("a score of ")[1].split(" ")[0].split(".")[0].split(",")[0].strip()
        if score:
            return int(score)
        else:
            return None
    else:
        return None

for file in files:
    items = []
    for item in jsonlines.open(file):
        for turn in ["judge1", "judge2"]:
            scores = []
            for rubric, judgement in item[turn].items():
                item[f"{turn}_{rubric}"] = filter_score(judgement)
        items.append(item)
        
    df = pd.DataFrame(items)
    dfs.append(df)

df = pd.concat(dfs)
df.shape

(480, 17)

In [29]:
df.set_index("generator").isna().mean(numeric_only=True, axis=1)

generator
alignment-handbook/zephyr-7b-dpo-full                          0.0625
alignment-handbook/zephyr-7b-dpo-full                          0.0000
alignment-handbook/zephyr-7b-dpo-full                          0.0625
alignment-handbook/zephyr-7b-dpo-full                          0.0000
alignment-handbook/zephyr-7b-dpo-full                          0.0000
                                                                ...  
heegyu/0507-zephyr-7b-sft-full-max-margin-1e-4-b0.5@epoch-1    0.0000
heegyu/0507-zephyr-7b-sft-full-max-margin-1e-4-b0.5@epoch-1    0.0000
heegyu/0507-zephyr-7b-sft-full-max-margin-1e-4-b0.5@epoch-1    0.0000
heegyu/0507-zephyr-7b-sft-full-max-margin-1e-4-b0.5@epoch-1    0.0000
heegyu/0507-zephyr-7b-sft-full-max-margin-1e-4-b0.5@epoch-1    0.0000
Length: 480, dtype: float64

In [30]:
cols = [f"{turn}_{rubric}" for turn in ["judge1", "judge2"] for rubric in ["helpfulness", "reasoning", "honesty", "factual_validity"] ]
df.groupby("generator").agg({col: "mean" for col in cols})

,judge1_helpfulness,judge1_reasoning,judge1_honesty,judge1_factual_validity,judge2_helpfulness,judge2_reasoning,judge2_honesty,judge2_factual_validity
generator,,,,,,,,
HuggingFaceH4/zephyr-7b-beta,3.207792,3.670886,3.421053,3.217949,1.289474,1.465753,1.432432,1.640000
alignment-handbook/zephyr-7b-dpo-full,3.564103,4.126582,3.506329,3.475000,1.328571,1.652778,1.514706,1.591549
alignment-handbook/zephyr-7b-sft-full,3.275000,3.480519,3.293333,153.275000,1.215190,1.220779,1.500000,1.354430
heegyu/0507-zephyr-7b-sft-full-max-margin-1e-4-b0.01@epoch-1,3.205128,4.000000,7.293333,3.300000,1.260274,1.416667,1.287671,1.474359
heegyu/0507-zephyr-7b-sft-full-max-margin-1e-4-b0.1@epoch-1,3.220779,3.493671,154.949367,3.065789,1.347222,1.440000,1.319444,1.454545
heegyu/0507-zephyr-7b-sft-full-max-margin-1e-4-b0.5@epoch-1,3.300000,3.786667,153.437500,3.303797,1.215190,1.263158,1.519481,1.307692


In [16]:
df[df[cols] > 0].groupby("generator").agg({col: "count" for col in cols})

TypeError: '>' not supported between instances of 'str' and 'int'